In [22]:
# imports 
import numpy as np
import pandas as pd

In [23]:
df= pd.read_csv('../data/train/diamonds_train_cleaned.csv', index_col=[0])
df.replace([np.inf, -np.inf], 0 , inplace=True) #In the cases where dimension = 0, the value of carat/dimensions is inf, which needs to be replaced by 0.
df.head()

,depth,table,x,y,z,price,carat,cut,color,clarity,city,xy,xz,zy,table_depth,carat/dimensons
0,62.4,58.0,6.83,6.79,4.25,4268,1.21,Premium,J,VS2,Dubai,1.005891,1.607059,0.625920,1.075862,0.006139
1,61.6,58.0,6.40,6.35,3.93,3513,1.02,Premium,J,VS2,Dubai,1.007874,1.628499,0.618898,1.062069,0.006386
2,62.3,58.0,5.86,5.80,3.63,1792,0.77,Premium,J,VS2,Dubai,1.010345,1.614325,0.625862,1.074138,0.006241
3,59.6,60.0,7.58,7.48,4.49,7553,1.51,Premium,J,VS2,Dubai,1.013369,1.688196,0.600267,0.993333,0.005931
4,60.2,62.0,5.40,5.33,3.23,1176,0.57,Premium,J,VS2,Dubai,1.013133,1.671827,0.606004,0.970968,0.006131


In [24]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_columns = list((df.select_dtypes(include=numerics)).columns)
categorical_columns = ['cut', 'color', 'clarity']

#Vamos a eleiminar las features con una correlación muy alta entre ellas, o con poca correlación con el precio:

numerical_columns.remove('price')
numerical_columns.remove('depth')
numerical_columns.remove('x')
numerical_columns.remove('zy')
numerical_columns.remove('y')
numerical_columns.remove('z')
numerical_columns

['table', 'carat', 'xy', 'xz', 'table_depth', 'carat/dimensons']

In [25]:
FEATS = numerical_columns + categorical_columns
TARGET = 'price'


In [26]:
df[FEATS]

,table,carat,xy,xz,table_depth,carat/dimensons,cut,color,clarity
0,58.0,1.21,1.005891,1.607059,1.075862,0.006139,Premium,J,VS2
1,58.0,1.02,1.007874,1.628499,1.062069,0.006386,Premium,J,VS2
2,58.0,0.77,1.010345,1.614325,1.074138,0.006241,Premium,J,VS2
3,60.0,1.51,1.013369,1.688196,0.993333,0.005931,Premium,J,VS2
4,62.0,0.57,1.013133,1.671827,0.970968,0.006131,Premium,J,VS2
...,...,...,...,...,...,...,...,...,...
40450,54.0,0.54,0.994307,1.602446,1.151852,0.005980,Ideal,F,IF
40451,54.0,0.53,0.994286,1.611111,1.146296,0.005969,Ideal,F,IF
40452,55.0,0.30,0.990783,1.598513,1.132727,0.005976,Ideal,F,IF
40453,55.0,0.26,0.981087,1.627451,1.107273,0.005808,Ideal,F,IF


In [27]:
# Apply label encoder to each column with categorical data
"""from sklearn.preprocessing import OneHotEncoder, LabelEncoder
label_encoder = LabelEncoder()
for col in categorical_columns:
    df[col] = label_encoder.fit_transform(df[col])
df.head()"""
data_one_hot_encoding = pd.get_dummies(df[FEATS])
data_one_hot_encoding.head()

,table,carat,xy,xz,table_depth,carat/dimensons,cut_Fair,cut_Good,cut_Ideal,cut_Premium,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,58.0,1.21,1.005891,1.607059,1.075862,0.006139,0,0,0,1,...,0,1,0,0,0,0,0,1,0,0
1,58.0,1.02,1.007874,1.628499,1.062069,0.006386,0,0,0,1,...,0,1,0,0,0,0,0,1,0,0
2,58.0,0.77,1.010345,1.614325,1.074138,0.006241,0,0,0,1,...,0,1,0,0,0,0,0,1,0,0
3,60.0,1.51,1.013369,1.688196,0.993333,0.005931,0,0,0,1,...,0,1,0,0,0,0,0,1,0,0
4,62.0,0.57,1.013133,1.671827,0.970968,0.006131,0,0,0,1,...,0,1,0,0,0,0,0,1,0,0


In [28]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [29]:
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(data_one_hot_encoding, df[TARGET], random_state=42)

In [38]:
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

model=RandomForestRegressor()


In [39]:
model.fit(data_train, target_train);

In [40]:
y_pred = model.predict(data_test)
y_pred

array([ 559.04      , 1243.56      , 4072.47666667, ..., 2475.01      ,
        521.31      ,  815.06      ])

In [41]:
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(target_test, y_pred)**0.5
rmse

536.357411884374

In [54]:
#Model prediction Test Dataframe (aka real prediction):
diamonds_train=pd.read_csv('../data/test/diamonds_test.csv', index_col=[0])
diamonds_train['xy'] = diamonds_train['x']/diamonds_train['y']
diamonds_train['xz'] = diamonds_train['x']/diamonds_train['z']
diamonds_train['zy'] = diamonds_train['z']/diamonds_train['y']
diamonds_train['table_depth'] = diamonds_train['table']/df['depth']
diamonds_train['carat/dimensons'] = diamonds_train ['carat'] / (diamonds_train['x']* diamonds_train['y'] * diamonds_train['z'])
diamonds_train.replace([np.inf, -np.inf], 0 , inplace=True)
diamonds_train


,carat,cut,color,clarity,depth,table,x,y,z,city,xy,xz,zy,table_depth,carat/dimensons
id,,,,,,,,,,,,,,,
0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67,Amsterdam,0.988115,1.585831,0.623090,0.961538,0.006279
1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat,0.988389,1.629187,0.606676,0.925325,0.006118
2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly,1.008197,1.614880,0.624317,0.979133,0.006359
3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly,0.993475,1.561538,0.636215,0.906040,0.006182
4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam,0.992141,1.583072,0.626719,0.963455,0.006098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,0.57,Ideal,E,SI1,61.9,56.0,5.35,5.32,3.30,Amsterdam,1.005639,1.621212,0.620301,0.919540,0.006069
13481,0.71,Ideal,I,VS2,62.2,55.0,5.71,5.73,3.56,New York City,0.996510,1.603933,0.621291,0.878594,0.006096
13482,0.70,Ideal,F,VS1,61.6,55.0,5.75,5.71,3.53,Tel Aviv,1.007005,1.628895,0.618214,0.892857,0.006040


In [55]:
"""for col in categorical_columns:
    diamonds_train[col] = label_encoder.fit_transform(diamonds_train[col])
diamonds_train.head()"""

data_one_hot_encoding_train = pd.get_dummies(diamonds_train[FEATS])
data_one_hot_encoding_train.head()

,table,carat,xy,xz,table_depth,carat/dimensons,cut_Fair,cut_Good,cut_Ideal,cut_Premium,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
id,,,,,,,,,,,,,,,,,,,,,
0,60.0,0.79,0.988115,1.585831,0.961538,0.006279,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,57.0,1.20,0.988389,1.629187,0.925325,0.006118,0,0,1,0,...,0,1,0,0,0,0,1,0,0,0
2,61.0,1.57,1.008197,1.614880,0.979133,0.006359,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
3,54.0,0.90,0.993475,1.561538,0.906040,0.006182,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,58.0,0.50,0.992141,1.583072,0.963455,0.006098,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [56]:
model.fit(data_one_hot_encoding,df[TARGET]); #Volvemos a entrenar el modelo con el datframe original (sin la separacion de train y test), para tener mas datos en el entreno

In [57]:
data_one_hot_encoding_train

,table,carat,xy,xz,table_depth,carat/dimensons,cut_Fair,cut_Good,cut_Ideal,cut_Premium,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
id,,,,,,,,,,,,,,,,,,,,,
0,60.0,0.79,0.988115,1.585831,0.961538,0.006279,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,57.0,1.20,0.988389,1.629187,0.925325,0.006118,0,0,1,0,...,0,1,0,0,0,0,1,0,0,0
2,61.0,1.57,1.008197,1.614880,0.979133,0.006359,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
3,54.0,0.90,0.993475,1.561538,0.906040,0.006182,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,58.0,0.50,0.992141,1.583072,0.963455,0.006098,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,56.0,0.57,1.005639,1.621212,0.919540,0.006069,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
13481,55.0,0.71,0.996510,1.603933,0.878594,0.006096,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
13482,55.0,0.70,1.007005,1.628895,0.892857,0.006040,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0


In [58]:
y_pred = model.predict(data_one_hot_encoding_train)
y_pred


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [59]:
y_pred=pd.DataFrame(y_pred, columns=['price'])
y_pred.reset_index(inplace=True)
y_pred.rename(columns={"index": "id"}, inplace= True)
y_pred

,id,price
0,0,4488.81
1,1,3593.34
2,2,1879.99
3,3,7438.66
4,4,1178.29
...,...,...
40420,40420,2697.45
40421,40421,2777.37
40422,40422,894.54
40423,40423,742.31


In [48]:
y_pred.to_csv('../data/prediction/price_prediction.csv', index= False)